In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [2]:
sites=pd.read_csv('shuffled_url.csv',index_col=0)

In [3]:
sites.head()

,domain,label
12313,kuhrsinholland.com/PAOAPL/secure-code54/home/4...,1
12738,www.cnwaxpd.com/js/?ref=us.battle.net/d3/en/index,1
34658,battle-login-en.sswow.asia/login/en/login.html...,1
12426,ok.ppok.asia/login/en/index.htm,1
27504,paktan.com/secure/index000.htm?www.nab.com.au/...,1


In [4]:
extracted=pd.read_csv('extracted.csv',index_col=0)

In [5]:
extracted.shape

(15000, 15)

In [6]:
extracted.head()

,Have_IP,Have_At,URL_Length,Redirection,https_Domain,hasHTTP,TinyURL,Prefix/Suffix,favicon,Request_URL,URL_of_anchor,links_in_tag,SFH,to_email,iFrame
0,0,0,0,0,0,1,0,0,1,1,1,1,1,1,1
1,0,0,0,0,0,1,0,0,1,1,1,1,1,1,1
2,0,0,0,0,0,1,0,1,1,1,1,1,1,1,1
3,0,0,0,0,0,1,0,0,1,1,1,1,1,1,1
4,0,0,0,0,0,1,0,0,1,1,1,1,1,1,1


In [7]:
sites=sites[0:15000]

In [8]:
sites.shape

(15000, 2)

In [9]:
sites=sites.reset_index()

In [10]:
sites.drop(['index'],axis=1,inplace=True)

In [11]:
sites.head()

,domain,label
0,kuhrsinholland.com/PAOAPL/secure-code54/home/4...,1
1,www.cnwaxpd.com/js/?ref=us.battle.net/d3/en/index,1
2,battle-login-en.sswow.asia/login/en/login.html...,1
3,ok.ppok.asia/login/en/index.htm,1
4,paktan.com/secure/index000.htm?www.nab.com.au/...,1


In [12]:
merged=pd.concat([sites,extracted],axis=1)

In [13]:
merged.head()

,domain,label,Have_IP,Have_At,URL_Length,Redirection,https_Domain,hasHTTP,TinyURL,Prefix/Suffix,favicon,Request_URL,URL_of_anchor,links_in_tag,SFH,to_email,iFrame
0,kuhrsinholland.com/PAOAPL/secure-code54/home/4...,1,0,0,0,0,0,1,0,0,1,1,1,1,1,1,1
1,www.cnwaxpd.com/js/?ref=us.battle.net/d3/en/index,1,0,0,0,0,0,1,0,0,1,1,1,1,1,1,1
2,battle-login-en.sswow.asia/login/en/login.html...,1,0,0,0,0,0,1,0,1,1,1,1,1,1,1,1
3,ok.ppok.asia/login/en/index.htm,1,0,0,0,0,0,1,0,0,1,1,1,1,1,1,1
4,paktan.com/secure/index000.htm?www.nab.com.au/...,1,0,0,0,0,0,1,0,0,1,1,1,1,1,1,1


In [14]:
merged.drop(['hasHTTP'],axis=1,inplace=True)

In [15]:
import json

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X=merged.drop(['domain','label'],axis=1)

In [18]:
X.head()

,Have_IP,Have_At,URL_Length,Redirection,https_Domain,TinyURL,Prefix/Suffix,favicon,Request_URL,URL_of_anchor,links_in_tag,SFH,to_email,iFrame
0,0,0,0,0,0,0,0,1,1,1,1,1,1,1
1,0,0,0,0,0,0,0,1,1,1,1,1,1,1
2,0,0,0,0,0,0,1,1,1,1,1,1,1,1
3,0,0,0,0,0,0,0,1,1,1,1,1,1,1
4,0,0,0,0,0,0,0,1,1,1,1,1,1,1


In [19]:
y=merged['label']

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
test_data = dict()
test_data['X_test'] = X_test.to_numpy().tolist()
test_data['y_test'] = y_test.to_numpy().tolist()
with open('testdata.json', 'w') as tdfile:
    json.dump(test_data, tdfile)
    print('Test Data written to testdata.json')

Test Data written to testdata.json


In [22]:
X_train=X_train.to_numpy()

In [23]:
y_train=y_train.to_numpy()

In [24]:
rfc=RandomForestClassifier(n_estimators=100)

In [25]:
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [26]:
predictions=rfc.predict(X_test)

In [27]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [28]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.81      0.58      0.67      1531
           1       0.66      0.86      0.75      1469

    accuracy                           0.71      3000
   macro avg       0.73      0.72      0.71      3000
weighted avg       0.74      0.71      0.71      3000



In [29]:
from sklearn.tree import _tree
def tree_to_json(tree):
    tree_ = tree.tree_
    feature_names = range(30)
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    def recurse(node):
        tree_json = dict()
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            tree_json['type'] = 'split'
            threshold = tree_.threshold[node]
            tree_json['threshold'] = "{} <= {}".format(feature_name[node], threshold)
            tree_json['left'] = recurse(tree_.children_left[node])
            tree_json['right'] = recurse(tree_.children_right[node])
        else:
            tree_json['type'] = 'leaf'
            tree_json['value'] = tree_.value[node].tolist()
        return tree_json

    return recurse(0)

def forest_to_json(forest):
    forest_json = dict()
    forest_json['n_features'] = forest.n_features_
    forest_json['n_classes'] = forest.n_classes_
    forest_json['classes'] = forest.classes_.tolist()
    forest_json['n_outputs'] = forest.n_outputs_
    forest_json['n_estimators'] = forest.n_estimators
    forest_json['estimators'] = [tree_to_json(estimator) for estimator in forest.estimators_]
    return forest_json

In [30]:
json.dump(forest_to_json(rfc), open('classifier.json', 'w'))